In [2]:
import random
import json
from datetime import datetime, timedelta

# Constants
VENUE_CAPACITIES = list(range(50, 321, 10))  # Venue capacities between 50 and 320
VENUES = [f"Venue {i+1}" for i in range(30)]  # Generate 30 venues
BLOCKS = [f"Block {chr(65 + i)}" for i in range(10)]  # Block A to Block J
STAFF_COUNT = 100  # Total number of invigilators
DEPARTMENTS = [f"Department {i+1}" for i in range(28)]  # 28 departments
TIME_SLOTS = {
    "08:00": "08:00 AM - 10:30 AM",
    "11:00": "11:00 AM - 01:00 PM",
    "14:00": "02:00 PM - 04:00 PM"
}
LUNCH_BREAK = "13:00"
START_DATE = datetime(2024, 7, 22)  # Start date for scheduling
WEEKDAYS = [0, 1, 2, 3, 4, 5]  # Monday to Saturday

# Generate weekday dates for a month
DATES = [
    (START_DATE + timedelta(days=i)).strftime("%Y-%m-%d")
    for i in range(30) if (START_DATE + timedelta(days=i)).weekday() in WEEKDAYS
]

# Example data for First Semester courses
COURSES = [
    {"course": f"Course {i+1}", "programme": f"Programme {i % 5 + 1}", "type": random.choice(["Written", "CBT"])}
    for i in range(100)
]

def generate_data():
    """
    Generate foundational data for the timetable.
    """
    data = {
        "venues": [],
        "invigilators": [],
        "courses": [],
        "time_slots": TIME_SLOTS,
        "lunch_break": LUNCH_BREAK,
        "dates": DATES,
        "metadata": {
            "version": "v1.0",
            "generated_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
    }

    # Generate venue data
    for venue, capacity, block in zip(VENUES, VENUE_CAPACITIES, random.choices(BLOCKS, k=len(VENUES))):
        data["venues"].append({
            "name": venue,
            "capacity": capacity,
            "block": block
        })

    # Generate invigilator data
    for i in range(1, STAFF_COUNT + 1):
        data["invigilators"].append({
            "name": f"Invigilator {i}",
            "department": random.choice(DEPARTMENTS)
        })

    # Include First Semester courses
    for course in COURSES:
        data["courses"].append({
            "course_name": course["course"],
            "program": course["programme"],
            "type": course["type"]
        })

    return data

# Save the generated data to a JSON file
if __name__ == "__main__":
    output_file = "data9.json"
    generated_data = generate_data()
    with open(output_file, "w") as f:
        json.dump(generated_data, f, indent=4)
    print(f"Data generated and saved to {output_file}")


Data generated and saved to data9.json


In [7]:
import json
import random
from datetime import datetime
from collections import defaultdict
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font, PatternFill

# Constants
TIME_SLOTS = {
    "08:00": "08:00 AM - 10:30 AM",
    "11:00": "11:00 AM - 01:00 PM",
    "14:00": "02:00 PM - 04:00 PM"
}
LUNCH_BREAK = "13:00"
DAY_ABBREVIATIONS = {
    "Monday": "MON", "Tuesday": "TUE", "Wednesday": "WED",
    "Thursday": "THU", "Friday": "FRI", "Saturday": "SAT"
}

# Helper Functions
def assign_invigilators(student_population):
    """Assign 1 invigilator per 40 students."""
    return max(1, student_population // 40)

def get_day_abbr(date_str):
    """Get the day abbreviation (e.g., MON, TUE)."""
    day = datetime.strptime(date_str, "%Y-%m-%d").strftime("%A")
    return DAY_ABBREVIATIONS.get(day, "")

# Scheduling Process
def generate_timetable(data):
    """
    Generate the exam timetable based on provided constraints.
    """
    timetable = []
    venue_usage = defaultdict(lambda: defaultdict(list))  # Track venue usage per day and time slot
    department_usage = defaultdict(lambda: defaultdict(list))  # Track department usage per day and time slot

    for course in sorted(data["courses"], key=lambda x: x.get("students", 0), reverse=True):
        course_name = course["course_name"]
        department = course["department"]
        student_population = course["students"]
        exam_type = course["type"]

        assigned_time_slot = None
        venue = None
        invigilators = assign_invigilators(student_population)

        # Try scheduling the course
        for date in data["dates"]:
            day_abbr = get_day_abbr(date)

            for time_key, time_label in TIME_SLOTS.items():
                # Check if a venue is available
                venue = next(
                    (v for v in data["venues"] if v["name"] not in venue_usage[date][time_key] and v["capacity"] >= student_population),
                    None
                )

                # Check department conflict
                if venue and department not in department_usage[date][time_key]:
                    assigned_time_slot = time_label
                    venue_usage[date][time_key].append(venue["name"])
                    department_usage[date][time_key].append(department)
                    break

            if assigned_time_slot:
                # Add to timetable
                timetable.append({
                    "Date": date,
                    "Day": day_abbr,
                    "Time": assigned_time_slot,
                    "Course": course_name,
                    "Department": department,
                    "Exam Type": exam_type,
                    "Venue": f"{venue['name']} (Block {venue['block']}, {venue['capacity']} seats)",
                    "Students Population": student_population,
                    "Invigilators": invigilators
                })
                break

    return timetable

# Formatting and Saving to Excel
def save_timetable_to_excel(timetable, file_name="Final_Exam_Timetable.xlsx"):
    """
    Save the timetable to an Excel file in the required format.
    """
    # Create Workbook
    wb = Workbook()
    ws = wb.active
    ws.title = "Exam Timetable"

    # Add Headers
    headers = ["Date", "Day", "Time", "Course", "Department", "Exam Type", "Venue", "Students Population", "Invigilators"]
    ws.append(headers)

    # Style Headers
    header_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    header_font = Font(bold=True)
    for col_num, header in enumerate(headers, start=1):
        cell = ws.cell(row=1, column=col_num)
        cell.fill = header_fill
        cell.font = header_font
        ws.column_dimensions[cell.column_letter].width = 25  # Adjust column width

    # Populate Rows
    for entry in timetable:
        ws.append([
            entry["Date"],
            entry["Day"],
            entry["Time"],
            entry["Course"],
            entry["Department"],
            entry["Exam Type"],
            entry["Venue"],
            entry["Students Population"],
            entry["Invigilators"]
        ])

    # Adjust alignment
    for row in ws.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True, vertical="center")

    # Save the file
    wb.save(file_name)
    print(f"Timetable successfully saved as {file_name}")

# Main Execution
if __name__ == "__main__":
    try:
        # Load Data
        with open("data9.json", "r") as f:
            data = json.load(f)

        # Generate Timetable
        timetable = generate_timetable(data)

        # Save to Excel
        save_timetable_to_excel(timetable)

    except Exception as e:
        print(f"Error: {e}")


Error: 'department'
